In [67]:
import numpy as np

# Question 5
The following problem arises in a large number of robotics and vision problems: Suppose p1, ..., pn are the 3D coordinates of n points located on a rigid body in three-space. Suppose further that q1, ..., qn are the 3D coordinates of these same points after the body has been translated and rotated by some unknown amount. Derive an algorithm in which SVD plays a central role for inferring the body’s translation and rotation. (You may assume that the coordinate values are precise
not noisy, but see comment and caution below.)

Show (in your pdf) that your algorithm works correctly by running it on some examples.

Comment: This problem requires some thought. There are different approaches. Although you can ﬁnd a solution on the web or in a vision text book, try to solve the problem yourself before looking at any such sources. Spend some time on the problem. It is good practice to develop your analytic skills. Feel free to discuss among yourselves. (As always, cite any sources, including discussions with others.)

Requirement: Your algorithm should make use of all the information available. True, in principle you only need three pairs of points – but if you use more points your solution will be more robust, something that might come in handy some day when you need to do this for real with noisy data.

Caution: A common mistake is to derive an algorithm that finds the best affine transformation, rather than the best rigid body transformation. Even though you may assume precise coordinate values, imagine how your algorithm would behave with noise. Your algorithm should still produce a rigid body transformation.

Hint: Suppose for a moment that both sets of points have the origin as centroid. Assemble all the points pi into a matrix P and all the points qi into another matrix Q. Now think about the relationship between P and Q. You may wish to ﬁnd a rigid body transformation that minimizes the sum of squared distances between the points qi and the result of applying the rigid body transformation to the points pi.

You may ﬁnd the following facts useful (assuming the dimensions are sensible):

‖x‖2 = xT x, xT RT y = T r(RxyT ).

[Here x and y are column vectors (e.g., 3D vectors) and R is a matrix (e.g., a 3 x 3 rotation matrix). The superscript T means transpose, so xT x is a number and xyT is a matrix. Also, Tr is the trace operator that adds up the diagonal elements of its square matrix argument.]

You will have more complicated expressions for x and y, involving the points pi and qi.

In [68]:
def generate_random_points(n: int) -> np.ndarray:
    """Generate random points in 3D space.

    Args:
        n (int): number of points to generate

    Returns:
        np.ndarray: 3 x n array of random points, where each column is a point
    """
    return np.random.rand(3, n)

In [69]:
def generate_random_rigid_body_transformation():
    """Generate a random rigid body transformation.

    Returns:
        R: 3 x 3 array representing the rotation matrix
        t: 3 x 1 array representing the translation vector
    """
    # Generate a random rotation matrix
    # Step 1: Generate a random axis (a unit vector)
    axis = np.random.randn(3)
    axis = axis / np.linalg.norm(axis)  # Normalize the vector to make it a unit vector

    # Step 2: Generate a random angle between 0 and 2π
    angle = np.random.uniform(0, 2 * np.pi)

    # Step 3: Use Rodrigues' rotation formula to construct the rotation matrix
    K = np.array(
        [[0, -axis[2], axis[1]], [axis[2], 0, -axis[0]], [-axis[1], axis[0], 0]]
    )  # Cross-product matrix of the axis

    I = np.eye(3)
    R = I + np.sin(angle) * K + (1 - np.cos(angle)) * (K @ K)

    assert np.allclose(
        np.linalg.det(R), 1
    ), "The generated matrix is not a rotation matrix"

    # Generate a random translation vector
    t = np.random.rand(3, 1)

    return R, t

In [70]:
def ICP(P: np.ndarray, Q: np.ndarray):
    """point-to-point iterative closest point algorithm with aligned point indexes

    Args:
        P (np.ndarray): source point cloud
        Q (np.ndarray): target point cloud
    """
    P = P.copy()
    Q = Q.copy()

    assert P.shape[0] == 3 and Q.shape[0] == 3, "Input point clouds must be 3D"
    assert (
        P.shape[1] == Q.shape[1]
    ), "Input point clouds must have the same number of points"

    # Center the point clouds
    P_centered = P - np.mean(P, axis=1, keepdims=True)
    Q_centered = Q - np.mean(Q, axis=1, keepdims=True)

    U, _, Vt = np.linalg.svd(P_centered @ Q_centered.T)

    if np.linalg.det(U @ Vt) < 0:
        Vt[-1] *= -1

    R = Vt.T @ U.T
    t = np.mean(Q, axis=1, keepdims=True) - R @ np.mean(P, axis=1, keepdims=True)

    return R, t

In [71]:
P = generate_random_points(100)
R, t = generate_random_rigid_body_transformation()
print("R =\n", R, "\nt =\n", t)
Q = R @ P + t

R =
 [[ 0.78760427  0.53794562  0.30048965]
 [-0.53690088  0.83842385 -0.09371707]
 [-0.30235237 -0.0875212   0.94916968]] 
t =
 [[0.16885856]
 [0.39316203]
 [0.61804859]]


In [72]:
R_ICP, t_ICP = ICP(P, Q)
print("R_ICP =\n", R_ICP, "\nt_ICP =\n", t_ICP)

R_ICP =
 [[ 0.78760427  0.53794562  0.30048965]
 [-0.53690088  0.83842385 -0.09371707]
 [-0.30235237 -0.0875212   0.94916968]] 
t_ICP =
 [[0.16885856]
 [0.39316203]
 [0.61804859]]


In [73]:
assert np.allclose(R, R_ICP, atol=1e-3), "The rotation matrix is not correct"
assert np.allclose(t, t_ICP, atol=1e-3), "The translation vector is not correct"